In [1]:
import pandas as pd 
import datetime

In [2]:
data1 = pd.read_excel(r"G:\work\logistica\stations\JDStations\stations\gaode/JDstations_beijing_gaode.xlsx")
df_station = data1[["JD_siteName","JD_siteLng_gaode","JD_siteLat_gaode","JD_siteDistrict_gaode","JD_siteCity_gaode"]]
# df_station.head()

In [3]:
data2_WL = pd.read_excel(r"G:\work\logistica\info\output\CHA\all\1901_JDWL.xlsx",header=None,skiprows=[0])
data2_XX = pd.read_excel(r"G:\work\logistica\info\output\CHA\all\1901_JDXX.xlsx",header=None,skiprows=[0])
data2 = pd.concat([data2_WL,data2_XX])
data2.columns = ['ORDER_ID','MAIL_NO','ROUTE_TIME','JD_deliveryman_name','JD_deliveryman_tel','ROUTE_TIME','LOGISTICS_COMPANY','JD_lastStation','JD_status']
df_log = data2.copy()

In [4]:
data3 = pd.read_csv(r"G:\work\basic\alter/1901_info_alter_gai.csv")

d:\python\envs\jupyterpy36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (11,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data3.shape

(714453, 48)

In [6]:
df_info = data3[data3["所在省"] == "北京"]

In [7]:
df_station.columns

Index(['JD_siteName', 'JD_siteLng_gaode', 'JD_siteLat_gaode',
       'JD_siteDistrict_gaode', 'JD_siteCity_gaode'],
      dtype='object')

In [8]:
df_log.columns

Index(['ORDER_ID', 'MAIL_NO', 'ROUTE_TIME', 'JD_deliveryman_name',
       'JD_deliveryman_tel', 'ROUTE_TIME', 'LOGISTICS_COMPANY',
       'JD_lastStation', 'JD_status'],
      dtype='object')

In [9]:
df_info.columns

Index(['订单编号', '订单状态', '订单子状态', '订单挂起原因', '安装失败原因', '订单来源', '店铺', '订单金额',
       '订单生成时间', '交易完成时间', '发货时间', '物流单回填时间', '入网用户姓名', '入网身份证号', '入网手机号',
       '收货人姓名', '收货地址', '收货人电话号码', '邮编', '所在省 / 市 / 县', '号码归属地', 'CPS推荐人',
       '订单备注', '物流单号', '物流签收时间', '承运商', '大小卡类型', '写卡渠道', '销售品编号', '销售品名称',
       '销售品类型', '套餐', '主号码', '副号码', '现金预存款', '靓号低消', '靓号等级', '靓号预存款',
       '营业厅送货方式', '是否线下模式', '是否线下转线上', '线下转线上原因', '营业厅送货iccid', '所在省', '所在市',
       '所在县', '号码归属省', '号码归属市'],
      dtype='object')

In [10]:
print("  df_info:",df_info.shape)
print("  df_log:",df_log.shape)
print("  df_station:",df_station.shape)

  df_info: (38648, 48)
  df_log: (103563, 9)
  df_station: (662, 5)


In [11]:
df_info['订单编号'] = df_info['订单编号'].map(lambda x:str(x))
df_log['ORDER_ID'] = df_log['ORDER_ID'].map(lambda x:str(x))

d:\python\envs\jupyterpy36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
# df1 = pd.merge(df_info,df_log,how="inner",left_on='订单编号',right_on='ORDER_ID')

In [13]:
# df1.shape

In [14]:
# df2 = pd.merge(df1,df_station,how="left",left_on="JD_lastStation",right_on="JD_siteName")

In [15]:
# df2.shape

In [16]:
df = pd.merge(pd.merge(df_info,df_log,how="inner",left_on='订单编号',right_on='ORDER_ID'),df_station,how="left",left_on="JD_lastStation",right_on="JD_siteName")

In [17]:
df.shape

(5573, 62)

In [18]:
df["订单生成时间"] = pd.to_datetime(df["订单生成时间"])
df["交易完成时间"] = pd.to_datetime(df["交易完成时间"])
df["发货时间"] = pd.to_datetime(df["发货时间"])
df["物流签收时间"] = pd.to_datetime(df["物流签收时间"])

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5573 entries, 0 to 5572
Data columns (total 62 columns):
订单编号                     5573 non-null object
订单状态                     5573 non-null object
订单子状态                    952 non-null object
订单挂起原因                   3 non-null object
安装失败原因                   0 non-null float64
订单来源                     5573 non-null object
店铺                       5573 non-null object
订单金额                     5573 non-null float64
订单生成时间                   5573 non-null datetime64[ns]
交易完成时间                   5015 non-null datetime64[ns]
发货时间                     5573 non-null datetime64[ns]
物流单回填时间                  0 non-null object
入网用户姓名                   5573 non-null object
入网身份证号                   5573 non-null object
入网手机号                    5573 non-null int64
收货人姓名                    5573 non-null object
收货地址                     5573 non-null object
收货人电话号码                  5573 non-null object
邮编                       4966 non-null object
所在省 

In [20]:
df["发货时效"] = round((df["发货时间"] - df["订单生成时间"]).dt.total_seconds() / 3600,0).map(lambda x:int(x))
df["签收时效"] = round((df["物流签收时间"] - df["发货时间"]).dt.total_seconds() / 3600,0).map(lambda x:int(x))
df["激活时效"] = round((df["交易完成时间"] - df["订单生成时间"]).dt.total_seconds() / 3600,0).fillna(-1).map(lambda x:int(x))

In [21]:
def get_sex(str1):
    #查看性别
    if str1 % 2 ==0:
        return '女'
    else:
        return '男'

In [22]:
df['性别'] = df["入网身份证号"].str.slice(16,17).map(lambda x:int(x)).apply(get_sex)

In [23]:
# 区分年龄段
df['生日'] = pd.to_datetime(df['入网身份证号'].str[6:14])

In [24]:
now_year =datetime.datetime.today().year #当前的年份
df['年龄']=now_year-df["生日"].dt.year

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5573 entries, 0 to 5572
Data columns (total 68 columns):
订单编号                     5573 non-null object
订单状态                     5573 non-null object
订单子状态                    952 non-null object
订单挂起原因                   3 non-null object
安装失败原因                   0 non-null float64
订单来源                     5573 non-null object
店铺                       5573 non-null object
订单金额                     5573 non-null float64
订单生成时间                   5573 non-null datetime64[ns]
交易完成时间                   5015 non-null datetime64[ns]
发货时间                     5573 non-null datetime64[ns]
物流单回填时间                  0 non-null object
入网用户姓名                   5573 non-null object
入网身份证号                   5573 non-null object
入网手机号                    5573 non-null int64
收货人姓名                    5573 non-null object
收货地址                     5573 non-null object
收货人电话号码                  5573 non-null object
邮编                       4966 non-null object
所在省 

In [40]:
df11["JD_lastStation"].value_counts()

北京水城营业部         71
北京沙河营业部         66
北京霍营乡营业部        45
北京辉煌国际营业部       44
北京歇甲村营业部        41
北京大红门营业部        40
北京宏福苑营业部        40
北京领航营业部         39
北京上地营业部         39
北京天宫院营业部        38
北京海淀营业部         38
北京酒仙桥营业部        37
北京首都机场营业部       37
北京花园路营业部        35
北京青塔营业部         35
北京回龙观营业部        35
北京安宁里营业部        34
北京东大桥营业部        33
北京航天城营业部        33
北京马池口营业部        33
北京踩河营业部         32
北京韩家川营业部        32
北京南沙滩营业部        32
北京总部基地营业部       31
北京永旺营业部         31
北京马连道营业部        31
北京龙华园营业部        31
北京吉祥里营业部        30
北京昌平营业部         30
北京瑞祥营业部         29
                ..
北京领峰营业部          3
北京密云营业部          3
北京模式口营业部         3
北京大石窝营业部         3
北京星河城营业部         3
北京宁馨苑营业部         3
北京行宫园营业部         2
北京太师屯营业部         2
北京大马庄营业部         2
北京梨园营业部          2
北京三槐堂营业部         2
北京北航营业部          2
北京礼贤营业部          2
北京月季园营业部         1
北京大学营业部          1
北京旭辉营业部          1
北京佛子庄营业部         1
北京万泉庄营业部         1
北京东柏街营业部         1
北京张山营营业部         1
北京大有庄营业部         1
北京双裕花园营业部   

In [31]:
# df.to_excel(r"G:\work\basic/BJ_JD_df2.xlsx",index=False)

False    5489
True       84
Name: JD_siteName, dtype: int64

In [32]:
df["JD_siteName"].isnull().value_counts()

False    5489
True       84
Name: JD_siteName, dtype: int64

In [37]:
import numpy as np
df11 = df[pd.notnull(df['JD_siteName'])]

In [38]:
df11.shape

(5489, 68)